### Тема “Предобработка текста с помощью Python”


Осуществим предобработку данных с Твиттера, чтобы отчищенный данные в дальнейшем использовать для задачи классификации. Данный датасет содержит негативные (label = 1) и нейтральные (label = 0) высказывания. Для работы объединим train_df и test_df.

In [1]:
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
import re
import json
import html
import nltk
import pickle

import pandas as pd

from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk import tokenize as tknz
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/levsolovev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/levsolovev/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/levsolovev/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
apostrophe_dict = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}



short_word_dict = {
"121": "one to one",
"a/s/l": "age, sex, location",
"adn": "any day now",
"afaik": "as far as I know",
"afk": "away from keyboard",
"aight": "alright",
"alol": "actually laughing out loud",
"b4": "before",
"b4n": "bye for now",
"bak": "back at the keyboard",
"bf": "boyfriend",
"bff": "best friends forever",
"bfn": "bye for now",
"bg": "big grin",
"bta": "but then again",
"btw": "by the way",
"cid": "crying in disgrace",
"cnp": "continued in my next post",
"cp": "chat post",
"cu": "see you",
"cul": "see you later",
"cul8r": "see you later",
"cya": "bye",
"cyo": "see you online",
"dbau": "doing business as usual",
"fud": "fear, uncertainty, and doubt",
"fwiw": "for what it's worth",
"fyi": "for your information",
"g": "grin",
"g2g": "got to go",
"ga": "go ahead",
"gal": "get a life",
"gf": "girlfriend",
"gfn": "gone for now",
"gmbo": "giggling my butt off",
"gmta": "great minds think alike",
"h8": "hate",
"hagn": "have a good night",
"hdop": "help delete online predators",
"hhis": "hanging head in shame",
"iac": "in any case",
"ianal": "I am not a lawyer",
"ic": "I see",
"idk": "I don't know",
"imao": "in my arrogant opinion",
"imnsho": "in my not so humble opinion",
"imo": "in my opinion",
"iow": "in other words",
"ipn": "I’m posting naked",
"irl": "in real life",
"jk": "just kidding",
"l8r": "later",
"ld": "later, dude",
"ldr": "long distance relationship",
"llta": "lots and lots of thunderous applause",
"lmao": "laugh my ass off",
"lmirl": "let's meet in real life",
"lol": "laugh out loud",
"ltr": "longterm relationship",
"lulab": "love you like a brother",
"lulas": "love you like a sister",
"luv": "love",
"m/f": "male or female",
"m8": "mate",
"milf": "mother I would like to fuck",
"oll": "online love",
"omg": "oh my god",
"otoh": "on the other hand",
"pir": "parent in room",
"ppl": "people",
"r": "are",
"rofl": "roll on the floor laughing",
"rpg": "role playing games",
"ru": "are you",
"shid": "slaps head in disgust",
"somy": "sick of me yet",
"sot": "short of time",
"thanx": "thanks",
"thx": "thanks",
"ttyl": "talk to you later",
"u": "you",
"ur": "you are",
"uw": "you’re welcome",
"wb": "welcome back",
"wfm": "works for me",
"wibni": "wouldn't it be nice if",
"wtf": "what the fuck",
"wtg": "way to go",
"wtgp": "want to go private",
"ym": "young man",
"gr8": "great"
}


emoticon_dict = {
":)": "happy",
":‑)": "happy",
":-]": "happy",
":-3": "happy",
":->": "happy",
"8-)": "happy",
":-}": "happy",
":o)": "happy",
":c)": "happy",
":^)": "happy",
"=]": "happy",
"=)": "happy",
"<3": "happy",
":-(": "sad",
":(": "sad",
":c": "sad",
":<": "sad",
":[": "sad",
">:[": "sad",
":{": "sad",
">:(": "sad",
":-c": "sad",
":-< ": "sad",
":-[": "sad",
":-||": "sad"
}

In [7]:
# Сохраним словари в файлы, для дальнейшего использования
with open('./apostrophe_dict.json', 'w', encoding="utf-8") as file:
    json.dump(apostrophe_dict, file)
with open('./short_word_dict.json', 'w', encoding="utf-8") as file:
    json.dump(short_word_dict, file)
with open('./emoticon_dict.json', 'w', encoding="utf-8") as file:
    json.dump(emoticon_dict, file)

In [8]:
df_test = pd.read_csv('./test_tweets.csv')

In [9]:
df_test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [10]:
df_test.shape

(17197, 2)

In [11]:
df_train = pd.read_csv('./train_tweets.csv')
df_train.shape

(31962, 3)

In [12]:
df_train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [13]:
combine_df = df_train.append(df_test, ignore_index = True, sort = False)
combine_df.head()

,id,label,tweet
0,1,0.0,@user when a father is dysfunctional and is s...
1,2,0.0,@user @user thanks for #lyft credit i can't us...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation


In [14]:
combine_df.tail()

,id,label,tweet
49154,49155,NaN,thought factory: left-right polarisation! #tru...
49155,49156,NaN,feeling like a mermaid ð #hairflip #neverre...
49156,49157,NaN,#hillary #campaigned today in #ohio((omg)) &am...
49157,49158,NaN,"happy, at work conference: right mindset leads..."
49158,49159,NaN,"my song ""so glad"" free download! #shoegaze ..."


In [15]:
combine_df['clean_tweet'] = combine_df['tweet'].map(lambda x: html.unescape(x))
combine_df.head()

,id,label,tweet,clean_tweet
0,1,0.0,@user when a father is dysfunctional and is s...,@user when a father is dysfunctional and is s...
1,2,0.0,@user @user thanks for #lyft credit i can't us...,@user @user thanks for #lyft credit i can't us...
2,3,0.0,bihday your majesty,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation,factsguide: society now #motivation


In [16]:
# Изменим регистр
combine_df['clean_tweet'] = combine_df['clean_tweet'].map(lambda x: x.lower())

In [17]:
# Заменим сокращения с апострофами 
for short_word, long_word in apostrophe_dict.items():
    combine_df['clean_tweet'] = combine_df['clean_tweet'].map(lambda x: re.sub(short_word, long_word, x))
combine_df.head()

,id,label,tweet,clean_tweet
0,1,0.0,@user when a father is dysfunctional and is s...,@user when a father is dysfunctional and is s...
1,2,0.0,@user @user thanks for #lyft credit i can't us...,@user @user thanks for #lyft credit i cannot u...
2,3,0.0,bihday your majesty,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation,factsguide: society now #motivation


In [18]:
# Заменим сокращения на их полные формы
for short_word, long_word in short_word_dict.items():
    combine_df['clean_tweet'] = combine_df['clean_tweet'].map(lambda x: re.sub(short_word, long_word, x))
combine_df.head()

,id,label,tweet,clean_tweet
0,1,0.0,@user when a father is dysfunctional and is s...,@youseare when a fatheare is dysfyounctional ...
1,2,0.0,@user @user thanks for #lyft credit i can't us...,@youseare @youseare thanks foare #lyft careedi...
2,3,0.0,bihday your majesty,bihday yoyouare majesty
3,4,0.0,#model i love u take with u all the time in ...,#model i love you take with you all the time...
4,5,0.0,factsguide: society now #motivation,factsgareinyouide: society now #motivation


In [19]:
# Заменим эмотиконы на пробелы
# Отдельно заменять не надо. Уберуться, когда заменим спец. символы на пробелы

In [20]:
# Удалим @user
combine_df['clean_tweet'] = combine_df['clean_tweet'].map(lambda x: re.sub("@[\w]*", ' ', x))
# Заменим пунктуацию на пробелы
combine_df['clean_tweet'] = combine_df['clean_tweet'].map(lambda x: re.sub('[^\w\s]', ' ', x))
# Заменим спец. символы на пробелы,
combine_df['clean_tweet'] = combine_df['clean_tweet'].map(lambda x: re.sub('[^a-zA-Z0-9]', ' ', x))
# Заменим числа на пробелы
combine_df['clean_tweet'] = combine_df['clean_tweet'].map(lambda x: re.sub('[^a-zA-Z]', ' ', x))
# Удалим из текста слова длиной в 1 символ
combine_df['clean_tweet'] = combine_df['clean_tweet'].map(lambda x: ' '.join([w for w in x.split() if len(w)>1]))
combine_df.head()

,id,label,tweet,clean_tweet
0,1,0.0,@user when a father is dysfunctional and is s...,when fatheare is dysfyounctional and is so sel...
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks foare lyft careedit cannot youse cayous...
2,3,0.0,bihday your majesty,bihday yoyouare majesty
3,4,0.0,#model i love u take with u all the time in ...,model love you take with you all the time in y...
4,5,0.0,factsguide: society now #motivation,factsgareinyouide society now motivation


In [21]:
# Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый столбец 'tweet_token'.
combine_df['tweet_token'] = combine_df['clean_tweet'].map(lambda x: tknz.word_tokenize(x))
combine_df.head()

,id,label,tweet,clean_tweet,tweet_token
0,1,0.0,@user when a father is dysfunctional and is s...,when fatheare is dysfyounctional and is so sel...,"[when, fatheare, is, dysfyounctional, and, is,..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks foare lyft careedit cannot youse cayous...,"[thanks, foare, lyft, careedit, can, not, yous..."
2,3,0.0,bihday your majesty,bihday yoyouare majesty,"[bihday, yoyouare, majesty]"
3,4,0.0,#model i love u take with u all the time in ...,model love you take with you all the time in y...,"[model, love, you, take, with, you, all, the, ..."
4,5,0.0,factsguide: society now #motivation,factsgareinyouide society now motivation,"[factsgareinyouide, society, now, motivation]"


In [22]:
# Удалим стоп-слова из токенов. Создадим столбец 'tweet_token_filtered' без стоп-слов.
stop_words = set(stopwords.words("english"))

In [23]:
combine_df['tweet_token_filtered'] = combine_df['tweet_token'].map(lambda words: [word for word in words if not word in stop_words])
combine_df.head()

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered
0,1,0.0,@user when a father is dysfunctional and is s...,when fatheare is dysfyounctional and is so sel...,"[when, fatheare, is, dysfyounctional, and, is,...","[fatheare, dysfyounctional, selfish, dareagare..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks foare lyft careedit cannot youse cayous...,"[thanks, foare, lyft, careedit, can, not, yous...","[thanks, foare, lyft, careedit, youse, cayouse..."
2,3,0.0,bihday your majesty,bihday yoyouare majesty,"[bihday, yoyouare, majesty]","[bihday, yoyouare, majesty]"
3,4,0.0,#model i love u take with u all the time in ...,model love you take with you all the time in y...,"[model, love, you, take, with, you, all, the, ...","[model, love, take, time, youare]"
4,5,0.0,factsguide: society now #motivation,factsgareinyouide society now motivation,"[factsgareinyouide, society, now, motivation]","[factsgareinyouide, society, motivation]"


In [24]:
# Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим столбец 'tweet_stemmed' после применения стемминга.
stemmer = PorterStemmer()

In [25]:
combine_df['tweet_stemmed'] = combine_df['tweet_token_filtered'].map(lambda words: [stemmer.stem(word) for word in words])
combine_df.head()

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed
0,1,0.0,@user when a father is dysfunctional and is s...,when fatheare is dysfyounctional and is so sel...,"[when, fatheare, is, dysfyounctional, and, is,...","[fatheare, dysfyounctional, selfish, dareagare...","[fathear, dysfyounct, selfish, dareagarein, ki..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks foare lyft careedit cannot youse cayous...,"[thanks, foare, lyft, careedit, can, not, yous...","[thanks, foare, lyft, careedit, youse, cayouse...","[thank, foar, lyft, careedit, yous, cayous, of..."
2,3,0.0,bihday your majesty,bihday yoyouare majesty,"[bihday, yoyouare, majesty]","[bihday, yoyouare, majesty]","[bihday, yoyouar, majesti]"
3,4,0.0,#model i love u take with u all the time in ...,model love you take with you all the time in y...,"[model, love, you, take, with, you, all, the, ...","[model, love, take, time, youare]","[model, love, take, time, youar]"
4,5,0.0,factsguide: society now #motivation,factsgareinyouide society now motivation,"[factsgareinyouide, society, now, motivation]","[factsgareinyouide, society, motivation]","[factsgareinyouid, societi, motiv]"


In [26]:
# Применим лемматизацию к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после применения лемматизации
lemmatizer = WordNetLemmatizer()
pos = wordnet.VERB

In [27]:
combine_df['tweet_lemmatized'] = combine_df['tweet_token_filtered'].map(lambda words: [lemmatizer.lemmatize(word, pos) for word in words])
combine_df.head()

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when fatheare is dysfyounctional and is so sel...,"[when, fatheare, is, dysfyounctional, and, is,...","[fatheare, dysfyounctional, selfish, dareagare...","[fathear, dysfyounct, selfish, dareagarein, ki...","[fatheare, dysfyounctional, selfish, dareagare..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks foare lyft careedit cannot youse cayous...,"[thanks, foare, lyft, careedit, can, not, yous...","[thanks, foare, lyft, careedit, youse, cayouse...","[thank, foar, lyft, careedit, yous, cayous, of...","[thank, foare, lyft, careedit, youse, cayouse,..."
2,3,0.0,bihday your majesty,bihday yoyouare majesty,"[bihday, yoyouare, majesty]","[bihday, yoyouare, majesty]","[bihday, yoyouar, majesti]","[bihday, yoyouare, majesty]"
3,4,0.0,#model i love u take with u all the time in ...,model love you take with you all the time in y...,"[model, love, you, take, with, you, all, the, ...","[model, love, take, time, youare]","[model, love, take, time, youar]","[model, love, take, time, youare]"
4,5,0.0,factsguide: society now #motivation,factsgareinyouide society now motivation,"[factsgareinyouide, society, now, motivation]","[factsgareinyouide, society, motivation]","[factsgareinyouid, societi, motiv]","[factsgareinyouide, society, motivation]"


In [28]:
with open('./combine_df.pkl', 'wb') as file:
    pickle.dump(combine_df, file)

In [29]:
combine_df.to_csv('./combine_df.csv', index=False, encoding='utf-8')